In [1]:
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import os, random
import numpy as np
from functions import select_dataset

In [2]:
testfile = pathlib.Path(r"C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData") / "FastCharge_000022_CH24_structure.json"
testfile.is_file()

True

In [3]:
df = pd.read_json(testfile)

In [4]:
df.head(5)

,@module,@class,barcode,protocol,channel_id,summary,cycles_interpolated,diagnostic_summary,diagnostic_interpolated,@version
cycle_index,beep.structure,ProcessedCyclerRun,el150800460614,2017-06-30_tests\20170630-4_9C_27per_4_75C.sdu,23,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",NaN,NaN,2020.10.19.20-8667111
discharge_capacity,beep.structure,ProcessedCyclerRun,el150800460614,2017-06-30_tests\20170630-4_9C_27per_4_75C.sdu,23,"[1.9485314999999992, 1.0623763, 1.065841, 1.06...","[0.504631323596709, 1.105153281993967, 1.56258...",NaN,NaN,2020.10.19.20-8667111
charge_capacity,beep.structure,ProcessedCyclerRun,el150800460614,2017-06-30_tests\20170630-4_9C_27per_4_75C.sdu,23,"[1.4362628, 1.0625434, 1.0656146, 1.0671674999...","[0.029696115076126003, 0.6200159021464431, 1.0...",NaN,NaN,2020.10.19.20-8667111
discharge_energy,beep.structure,ProcessedCyclerRun,el150800460614,2017-06-30_tests\20170630-4_9C_27per_4_75C.sdu,23,"[6.165954099999999, 3.2340322, 3.2391577, 3.24...",NaN,NaN,NaN,2020.10.19.20-8667111
charge_energy,beep.structure,ProcessedCyclerRun,el150800460614,2017-06-30_tests\20170630-4_9C_27per_4_75C.sdu,23,"[4.7377286, 3.7020485, 3.705625499999999, 3.71...",NaN,NaN,NaN,2020.10.19.20-8667111


In [38]:
%%time

directory = r"C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData"
savepath1 = r"C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\ML_Github\data\interim\summary"
savepath2 = r"C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\ML_Github\data\interim\cycles_interpolated\cycle10_discharge"
savepath3 = r"C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\ML_Github\data\interim\cycles_interpolated\cycle100_discharge"
savepath4 = r"C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\ML_Github\data\interim\cycles_interpolated\DeltaQ100_10_discharge"


# iterate over files in that directory
i = 0
for filename in os.listdir(directory)[101:]:
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        print(f)
        # Make dataframe of file and add the summary column to list
        df = pd.read_json(f)
        newfilename = filename[0:-15]
        
        # Save csv file with summary data
        summary = select_dataset(df, 'summary')
        summary.to_csv(os.path.join(savepath1, newfilename + '_summary.csv'))
        
        # Save csv file with cycle 10 and 100 discharge data
        cycles_interpolated = select_dataset(df, 'cycles_interpolated')
        cycles_interpolated['cycle_life'] = cycles_interpolated.index[-1]
        
        cycle10 = cycles_interpolated.loc[9,:]
        cycle100 = cycles_interpolated.loc[99,:]
        
        cycle10_discharge = cycle10.loc[cycle10['step_type'] == 'discharge']
        cycle100_discharge = cycle100.loc[cycle100['step_type'] == 'discharge']
        
        cycle10_discharge.to_csv(os.path.join(savepath2, newfilename + '_cycle10.csv'))
        cycle100_discharge.to_csv(os.path.join(savepath3, newfilename + '_cycle100.csv'))
        
        DeltaQ100_10 = pd.DataFrame({'discharge_capacity': cycle100_discharge['discharge_capacity'].values\
                                     - cycle10_discharge['discharge_capacity'].values,\
                                     'cycle_life': cycle10_discharge['cycle_life']})
        
        DeltaQ100_10.to_csv(os.path.join(savepath4, newfilename + '_DeltaQ100_10.csv'), index=False)

C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData\FastCharge_000046_CH31_structure.json
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (16, 1)
Shape of selected unpacked dataset: (891, 15)
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (8, 1)
Shape of selected unpacked dataset: (1784000, 7)
C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData\FastCharge_000046_CH32_structure.json
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (16, 1)
Shape of selected unpacked dataset: (1015, 15)
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (8, 1)
Shape of selected unpacked dataset: (2030000, 7)
C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData\FastCharge_000047_CH21_structure.json
Shape of selected packed dataset: (21, 1)
Sha

Shape of selected unpacked dataset: (1092000, 7)
C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData\FastCharge_000060_CH29_structure.json
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (16, 1)
Shape of selected unpacked dataset: (522, 15)
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (8, 1)
Shape of selected unpacked dataset: (1044000, 7)
C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData\FastCharge_000061_CH43_structure.json
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (16, 1)
Shape of selected unpacked dataset: (494, 15)
Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (8, 1)
Shape of selected unpacked dataset: (988000, 7)
C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData\FastCharge_000062_CH47_structure.jso

In [3]:
directory = r"C:\Users\ife12216\OneDrive - Institutt for Energiteknikk\Documents\Masteroppgave\MLData"

testfile = pathlib.Path(directory) / ("FastCharge_000002_CH18_structure.json")
testfile.is_file()

True

In [4]:
df_test = pd.read_json(testfile)
cyc_int = select_dataset(df_test, "cycles_interpolated")

Shape of selected packed dataset: (21, 1)
Shape of selected packed dataset without NaNs: (8, 1)
Shape of selected unpacked dataset: (1658000, 7)


In [5]:
cyc_int

,discharge_capacity,charge_capacity,voltage,temperature,current,internal_resistance,step_type
cycle_index,,,,,,,
0,0.550896,0.091248,2.8,31.964939,-1.383596,0.015512,discharge
0,0.62776,0.171367,2.800701,32.150776,-1.632531,0.015512,discharge
0,0.704625,0.251486,2.801401,32.336613,-1.881466,0.015512,discharge
0,0.78149,0.331604,2.802102,32.522449,-2.1304,0.015512,discharge
0,0.858354,0.411723,2.802803,32.708286,-2.379335,0.015512,discharge
...,...,...,...,...,...,...,...
828,0.879694,1.757092,3.551798,31.376076,0.109992,0.018316,charge
828,0.879694,1.758858,3.567995,31.36191,0.109992,0.018316,charge
828,0.879694,1.760624,3.587237,31.429064,0.110005,0.018316,charge


In [16]:
cyc_int['cycle_life'] = cyc_int.index[-1]

In [17]:
cyc_int

,discharge_capacity,charge_capacity,voltage,temperature,current,internal_resistance,step_type,cycle_life
cycle_index,,,,,,,,
0,0.550896,0.091248,2.8,31.964939,-1.383596,0.015512,discharge,828
0,0.62776,0.171367,2.800701,32.150776,-1.632531,0.015512,discharge,828
0,0.704625,0.251486,2.801401,32.336613,-1.881466,0.015512,discharge,828
0,0.78149,0.331604,2.802102,32.522449,-2.1304,0.015512,discharge,828
0,0.858354,0.411723,2.802803,32.708286,-2.379335,0.015512,discharge,828
...,...,...,...,...,...,...,...,...
828,0.879694,1.757092,3.551798,31.376076,0.109992,0.018316,charge,828
828,0.879694,1.758858,3.567995,31.36191,0.109992,0.018316,charge,828
828,0.879694,1.760624,3.587237,31.429064,0.110005,0.018316,charge,828


In [32]:
cycle10 = cyc_int.loc[9,:]
cycle100 = cyc_int.loc[99,:]

In [20]:
Qd_c10 = cycle10.loc[cycle10['step_type'] == 'discharge']
Qd_c100 = cycle100.loc[cycle100['step_type'] == 'discharge']
#DeltaQ100_10 = pd.DataFrame(Qd_c100.values - Qd_c10.values)

In [22]:
Qd_c10

,discharge_capacity,charge_capacity,voltage,temperature,current,internal_resistance,step_type,cycle_life
cycle_index,,,,,,,,
9,0.996059,1.067734,2.8,34.752182,-4.400151,0.015295,discharge,828
9,0.995862,1.067734,2.800701,34.749432,-4.40015,0.015295,discharge,828
9,0.995653,1.067734,2.801401,34.746502,-4.400133,0.015295,discharge,828
9,0.995444,1.067734,2.802102,34.743568,-4.400115,0.015295,discharge,828
9,0.995235,1.067734,2.802803,34.740635,-4.400098,0.015295,discharge,828
...,...,...,...,...,...,...,...,...
9,0.000009,1.067734,3.497197,31.146788,-4.21298,0.015295,discharge,828
9,0.000009,1.067734,3.497898,31.146948,-4.202245,0.015295,discharge,828
9,0.000009,1.067734,3.498599,31.147108,-4.19151,0.015295,discharge,828


In [29]:
DeltaQ100_10 = pd.DataFrame({'discharge_capacity': Qd_c100['discharge_capacity'].values - Qd_c10['discharge_capacity'].values,\
                             'cycle_life': Qd_c100['cycle_life']})
DeltaQ100_10

,discharge_capacity,cycle_life
cycle_index,,
99,-0.023634,828
99,-0.023672,828
99,-0.023699,828
99,-0.023726,828
99,-0.023753,828
...,...,...
99,-0.000003,828
99,-0.000003,828
99,-0.000003,828
